# Logistic Regression Experiment


In this notebook, our objective is to replicate the **Logistic Map** experiment presented in the paper {footcite:t}`Kostic2023DPNets`, using the `kooplearn` library. This experiment explores the complexities of learning a representation of the noisy logistic map, a one-dimensional dynamical system characterized by:

$$
    x_{t + 1} = (rx_{t}(1 - x_{t}) + \xi_{t}) \mod 1.
$$

Here $\xi_{t}$ is an i.i.d trigonometric noise term with density $\propto \cos^{N}(x)$ with $N$ being an even integer, and $r$ is a positive number characterizing the logistic map. In this experiment, $r$ is set to 4 for which we can derive the exact solution.

In particular, the transfer operator for this problem has rank $N + 1$, it is non-normal, and its eigenvalues and eigenfunctions can be computed exactly. For more information about the transfer operator properties of this dynamical system, see {footcite:t}`Kostic2022`.

It is noteworthy that because the associated transfer operator, $\mathcal{T}$, is *non-normal*, learning its spectral decomposition is challenging {footcite:t}`Kostic2023SpectralRates`. Therefore, this experiment would be a good choice to evaluate the performance of learned representation.
Especially since we know $\mathcal{T}$ can be computed exactly here we sidestep the problem of operator regression and just focus on evaluating the quality of the learned representation.

In this regard, we first check whether the `kooplearn` is installed or not, and if not install it.
## Setup `kooplearn`+ Import libraries

In [1]:
import importlib

for module in ['kooplearn']: # !! Add here any additional module that you need to install on top of kooplearn
    try:
        importlib.import_module(module)
    except ImportError:
      if module == 'kooplearn':
        module = 'kooplearn[full]'
      %pip install -q {module}

In [2]:
import numpy as np
import torch
from functools import partial
from torch.utils.data import DataLoader

In [3]:
# All hyperparameters
# {N: 20
# num_rng_seeds: 20
# num_train: 16385 #batch_size*4 + 1
# num_val: 10000
# num_test: 10000
# feature_dim: 8
# batch_size: 8192
# layer_dims: [64, 128, 64]
# max_epochs: 500
# trial_budget: 50
# learning_rate: 5e-5}

## Building the Dataset

Since in this experiment we want to learn the logistic map, we should build our dataset first. To this purpose, we follow these steps respectively:

- Define our dynamical system;
- Sample from this system to generate the training, validation, and test trajectories;
- Use {meth}`kooplearn.data.traj_to_contexts` to transform the trajectories in context windows (see [Kooplearn’s data paradigm](kooplearn_data_paradigm) for more information);
- Create a DataLoader to use PyTorch

### Defining the Noisy Logistic Map

Before defining the dynamical system, we should determine the number of training, validation, and test samples. As we intended to reproduce the paper's result we set the training number and batch size as they reported in the appendix `F.1`. Here, as we know the analytical solution of the problem, we set the number of test samples arbitrarily.  However, we will not use these samples for evaluation of the learned representation in this problem. Also, we will not perform hyperparameter tuning and just set the hyperparameters given in the paper.

In [4]:
# Defining the number of samples for each data split
batch_size = 8192
n_train_samples = 16385 #batch_size*2 + 1
n_val_samples = 10000
n_test_samples = 10000

In the `kooplearn` library, you can easily import {class}`kooplearn.datasets.LogisticMap` and generate the samples. For defining the dynamical system we can instantiate an object from the class, {class}`kooplearn.datasets.LogisticMap`, which takes $r$, $N$, and a random seed just for the sake of reproducibility.


In [5]:
from kooplearn.datasets import LogisticMap

# Model Initialization
N_exp = 20 # Exponent of the trigonometric noise
logmap = LogisticMap(N = N_exp, rng_seed = 0) # Setting the rng_seed for reproducibility

### Generating training, validation, and test trajectories

After instantiating from {class}`kooplearn.datasets.LogisticMap`, we can use this object to generate the samples calling the `sample` method. This function takes as inputs the initial condition/point, *$X_0$*, and the number of samples, *num_samples*, and returns the sampled trajectory starting from *$X_0$* and continuing for the number of samples. Thus, in total, it has the length *num_samples + 1*. After sampling our trajectory, we can split it to the training, validation and test samples.

In [6]:
# Data pipeline
sample_traj = logmap.sample(0.5, n_train_samples + n_val_samples + n_test_samples) #initalization point, X0 = 0.5 and T = length of trajectory.

dataset = {
    "train": sample_traj[: n_train_samples],
    "validation": sample_traj[n_train_samples : n_train_samples + n_val_samples],
    "test": sample_traj[n_train_samples + n_val_samples :],
}

train_data = torch.from_numpy(dataset["train"]).float()
val_data = torch.from_numpy(dataset["validation"]).float()

### From trajectories to context windows

As explained in [the documentation page of kooplearn's data paradigm](kooplearn_data_paradigm), `kooplearn` expects data organized into context windows. Broadly speaking, a context window is a (usually short) sequence of consecutive observations of the system. A context window encloses the 'past' in its _lookback window_ and the 'future' in its _lookforward window_.

Kooplearn provides simple utility functions to create context windows out of trajectories, such as {meth}`kooplearn.data.traj_to_contexts`. By default, {meth}`kooplearn.data.traj_to_contexts` will create contexts of length 2, as the minimal amount of frames to learn a dynamical relation is at least one for both the lookback and lookforward windows.

In [15]:
from kooplearn.data import traj_to_contexts

contexts = {k: traj_to_contexts(v) for k, v in dataset.items()} # Converting the trajectories to contexts
for split, ds in contexts.items():
    print(f"{split.capitalize()} contexts have shape {ds.shape}: {len(ds)} contexts of length {ds.context_length} with {ds.shape[2]} features each")

Train contexts have shape (16384, 2, 1): 16384 contexts of length 2 with 1 features each
Validation contexts have shape (9999, 2, 1): 9999 contexts of length 2 with 1 features each
Test contexts have shape (10000, 2, 1): 10000 contexts of length 2 with 1 features each


### Build Pytorch DataLoader

Here, we can simply use `Dataloader` function of PyTorch and pass the data and batch size as inputs.

N.B.: In order for the dataloaders to work properly with the context window datasets, we need to create the dataloader by passing the {meth}`kooplearn.nn.data.collate_context_dataset` to the dataloader as `DataLoader(..., collate_fn=collate_context_dataset)`.

In [21]:
from kooplearn.nn.data import collate_context_dataset

train_dl = DataLoader(contexts['train'], batch_size= batch_size, shuffle=True, collate_fn=collate_context_dataset)
val_dl = DataLoader(contexts['validation'], batch_size=len(contexts['validation']), shuffle=False, collate_fn=collate_context_dataset)

## DPNets Training Algorithm

To run the DPNets training algorithm, we should give DNNs, optimizer, number of training steps, and metric loss parameter as inputs of algorithm. Thus, here first we set the hyperparameters as reported in the original paper. And in the following parts describe:

- Sinusoidal Embedding
- Building NN Using PyTorch
- Build DPNet with Kaiming initialization
- (Hyperparameter Optimization for finding best learning rate with lightning Library)

In [72]:
# Init report dict
dl_reports = {}

opt = torch.optim.Adam
max_epochs = 100
num_rng_seeds = 20
layer_dims = [64, 128, 64]
metric_deformation_coeff = 1
feature_dim = 7

trainer_kwargs = {
    "accelerator": "gpu",
    "devices": 1,
    "max_epochs": max_epochs,
    'log_every_n_steps': 3,
    'enable_model_summary': False
}

### Sinusoidal Embedding

The logistic map is defined only in the interval [0, 1], and crucially is periodic. Therefore, featurizing the signal with trigonometric functions constructs models which account for this periodicity by design. We simply considering sin(2$\mathcal{\pi}$x) and cos(2$\mathcal{\pi}$x) as embedding.

In [73]:
class SinusoidalEmbedding(torch.nn.Module):
    def __init__(self):
        super().__init__()

    def forward(self, x):
        # Assuming x is in [0, 1]
        x = 2 * torch.pi * x
        return torch.cat([torch.sin(x), torch.cos(x)], dim=-1)

### Creting MLP using PyTorch

Now we use PyTorch library to define our neural networks with the specified hyperparameters:

In [74]:
class SimpleMLP(torch.nn.Module):
    def __init__(
        self, feature_dim: int, layer_dims: list[int], activation=torch.nn.LeakyReLU
    ):
        super().__init__()
        self.activation = activation
        lin_dims = ([2] + layer_dims + [feature_dim])  # The 2 is for the sinusoidal embedding

        layers = []

        for layer_idx in range(len(lin_dims) - 2):
            layers.append(torch.nn.Linear(lin_dims[layer_idx], lin_dims[layer_idx + 1], bias=False))
            layers.append(activation())

        layers.append(torch.nn.Linear(lin_dims[-2], lin_dims[-1], bias=True))

        self.layers = torch.nn.ModuleList(layers)
        self.sin_embedding = SinusoidalEmbedding()

    def forward(self, x):
        # Sinusoidal embedding
        x = self.sin_embedding(x).float()
        # MLP
        for layer in self.layers:
            x = layer(x)
        return x

### Building DPNet with Kaiming Initialization

Now we can define the DPNet using {class}`kooplearn.models.feature_maps.DPNet`. It takes both NNs(one for $\mathcal{Ψ_w}$ and one for$\mathcal{Ψ'_w}$), optimizer, trainer(Which is an initialized `Lightning` trainer), the metric deformation loss coefficient, a boolean variable to see wheter to compute the VAMP score with centered covariances and seed.

In [75]:
from kooplearn.models.feature_maps.nn import NNFeatureMap
from kooplearn.nn import DPLoss
import lightning

trainer = lightning.Trainer(**trainer_kwargs)
net_kwargs = {"feature_dim": feature_dim, "layer_dims": layer_dims}
DPNets = {'loss_fn': DPLoss, 'loss_kwargs': {'relaxed': False, 'metric_deformation': metric_deformation_coeff, 'center_covariances': False}}
seed = 42

# Defining the model
dpnet_fmap = NNFeatureMap(
    SimpleMLP,
    DPNets['loss_fn'],
    opt,
    trainer,
    encoder_kwargs = net_kwargs,
    loss_kwargs = DPNets['loss_kwargs'],
    optimizer_kwargs = {"lr": 5e-5},
    lagged_encoder = SimpleMLP,
    lagged_encoder_kwargs = net_kwargs,
    seed = seed)

GPU available: True (cuda), used: True
TPU available: False, using: 0 TPU cores
IPU available: False, using: 0 IPUs
HPU available: False, using: 0 HPUs
Seed set to 42


Similar to the original paper we will use the Kaiming Initialization for weights with considering the reaky ReLU as activation function except the first layer and bias. We initialize the bias terms zero, and for the first layer we will use uniform distribution between [-1,1].

After creating DPNet object we can easily try to train the model with training samples with {meth}.fit which takes training or validation dataloader as its single input.

In [76]:
def kaiming_init(model):
    for p in model.parameters():
        psh = p.shape
        if len(psh) == 2:  # Linear layers
            _, in_shape = psh
            if in_shape == 2:  # Initial layer
                torch.nn.init.uniform_(p, -1, 1)
            else:
                acname = model.activation.__name__.lower()
                if acname == "leakyrelu":
                    acname = "leaky_relu"
                torch.nn.init.kaiming_uniform_(p, a=1, nonlinearity=acname)
        else:  # Bias
            torch.nn.init.zeros_(p)

In [77]:
torch.manual_seed(seed)
kaiming_init(dpnet_fmap.lightning_module.encoder)
kaiming_init(dpnet_fmap.lightning_module.lagged_encoder)
dpnet_fmap.fit(train_dl)

LOCAL_RANK: 0 - CUDA_VISIBLE_DEVICES: [0,1]


Fitting NNFeatureMap. Lookback window length set to 1


Training: |          | 0/? [00:00<?, ?it/s]

`Trainer.fit` stopped: `max_epochs=100` reached.


## Evaluation Metrics
Now it is time to evaluate the performance of model. To do this, for this specific example we will use two different metrics that are used in the paper: (i) the optimality gap, and (ii) the spectral error.

**Optimality Gap**

The definition of the optimality gap in the paper is $
\sum_{i = 1}^{3} \sigma_{i}^{2}(\tau) - \mathcal{P}^{0}(w)$, which informs on how close one is to capture the best rank-r approximation of the transfer operator, $\mathcal{T}$.

**Spectral Error**

The spectral error is calculated by $ max_i \ min_j \left | \lambda_i(\mathcal{P}_{\mathcal{H}}\tau_{|\mathcal{H}}) - \lambda_j(\tau) \right |$. This formula measures how well the true eigenbalues of $\mathcal{T}$ can be recovered within the representation space $\mathcal{H_w}$.


In order to calculate the above metrics for evaluation the performance of the learned representation we should first calculate populational covariances and cross-covariances for the learned feature map.


The population_covs function computes population covariance and cross-covariance matrices based on given feature and logistic maps. It begins by generating input data points and then calculates eigenvalues and left eigenfunctions of the logistic map. Subsequently, it normalizes the Perron eigenvector and evaluates the feature map on the input data to obtain a feature matrix. Covariance and cross-covariance matrices are then computed using the feature matrix and normalized Perron eigenvector.

On the other hand, the evaluate_representation function assesses the performance of the obtained representation. It computes the OLS estimator, estimates eigenvalues, and evaluates metrics such as the optimality gap and spectral error to provide insights into the model's quality and accuracy.

In [78]:
from kooplearn.abc import FeatureMap
from scipy.integrate import romb

def population_covs(feature_map: FeatureMap, logistic: LogisticMap, pow_of_two_k: int = 12):
    """Computes the population covariance and cross-covariance"""
    x = np.linspace(0, 1, 2**pow_of_two_k + 1)[:, None]
    vals, lv = logistic.eig(eval_left_on=x) # returns eigenvalues and left eigenfunctions
    perron_eig_idx = np.argmax(np.abs(vals))
    pi = lv[:, perron_eig_idx] #eigenfunction correspond with the max eigenvalue
    assert np.isreal(pi).all()
    pi = pi.real
    pi = pi / romb(pi, dx=1 / 2**pow_of_two_k)  # Normalization of π
    # Evaluating the feature map
    phi = feature_map(x)  # [2**pow_of_two_k + 1, d]
    # Covariance
    cov_unfolded = (
        phi.reshape(2**pow_of_two_k + 1, -1, 1)
        * phi.reshape(2**pow_of_two_k + 1, 1, -1)
        * pi.reshape(-1, 1, 1)
    )
    cov = romb(cov_unfolded, dx=1 / 2**pow_of_two_k, axis=0)
    # Cross-covariance
    alphas = np.stack(
        [logistic.noise_feature_composed_map(x, n) for n in range(logistic.N + 1)],
        axis=1,
    )
    betas = np.stack(
        [logistic.noise_feature(x, n) for n in range(logistic.N + 1)], axis=1
    )

    cov_alpha_unfolded = (
        phi.reshape(2**pow_of_two_k + 1, -1, 1)
        * alphas.reshape(2**pow_of_two_k + 1, 1, -1)
        * pi.reshape(-1, 1, 1)
    )
    cov_beta_unfolded = phi.reshape(2**pow_of_two_k + 1, -1, 1) * betas.reshape(
        2**pow_of_two_k + 1, 1, -1
    )

    cov_alpha = romb(cov_alpha_unfolded, dx=1 / 2**pow_of_two_k, axis=0)
    cov_beta = romb(cov_beta_unfolded, dx=1 / 2**pow_of_two_k, axis=0)

    cross_cov = cov_alpha @ (cov_beta.T)
    return cov, cross_cov

In [79]:
from kooplearn._src.metrics import directed_hausdorff_distance
from kooplearn._src.utils import topk

def evaluate_representation(feature_map: FeatureMap, logistic_map: LogisticMap):
    report = {}
    # Compute OLS estimator
    cov, cross_cov = population_covs(feature_map, logistic_map)
    OLS_estimator = np.linalg.solve(cov, cross_cov)

    # Eigenvalue estimation
    OLS_eigs = np.linalg.eigvals(OLS_estimator)
    top_eigs = topk(np.abs(OLS_eigs), 3)
    OLS_eigs = OLS_eigs[top_eigs.indices]

    true_eigs = logistic_map.eig()
    top_eigs = topk(np.abs(true_eigs), 3)
    true_eigs = true_eigs[top_eigs.indices]

    report["hausdorff-distance"] = directed_hausdorff_distance(OLS_eigs, true_eigs)
    # VAMP2-score
    M = np.linalg.multi_dot(
        [
            np.linalg.pinv(cov, hermitian=True),
            cross_cov,
            np.linalg.pinv(cov, hermitian=True),
            cross_cov.T,
        ]
    )
    feature_dim = cov.shape[0]
    report["optimality-gap"] = np.sum(logistic_map.svals()[:feature_dim] ** 2) - np.trace(M)

    # Feasibility
    # report["feasibility-gap"] = np.linalg.norm(cov - np.eye(feature_dim), ord=2)
    # report["estimator-eigenvalues"] = OLS_eigs
    # report["covariance-eigenvalues"] = np.linalg.eigvalsh(cov)
    return report

Now we can assess the performace of the learned representation based on optimality gap and spectral error or hausdorff distance:

In [80]:
evaluate_representation(dpnet_fmap, logmap)

{'hausdorff-distance': 0.05323019211157242,
 'optimality-gap': 0.7968271997174887}

As we saw the result is aligned with the reported results in the paper, however we only run for one seed. Therefore, to establish a more meaningful statistical comparison between different models which tries to learn the representation we run each models for 20 independent runs and report mean and standard deviation.

## Evaluating DPNet

Now we run the DPNet over 20 independent runs:

In [82]:
relaxed = False
report = []

for rng_seed in range(num_rng_seeds):

    trainer = lightning.Trainer(**trainer_kwargs)
    net_kwargs = {"feature_dim": feature_dim, "layer_dims": layer_dims}

    # Defining the model
    dpnet_fmap = NNFeatureMap(
        SimpleMLP,
        DPNets['loss_fn'],
        opt,
        trainer,
        encoder_kwargs = net_kwargs,
        loss_kwargs = DPNets['loss_kwargs'],
        optimizer_kwargs = {"lr": 5e-5},
        lagged_encoder = SimpleMLP,
        lagged_encoder_kwargs = net_kwargs,
        seed = rng_seed)

    # Init
    torch.manual_seed(rng_seed)
    kaiming_init(dpnet_fmap.lightning_module.encoder)
    kaiming_init(dpnet_fmap.lightning_module.lagged_encoder)

    dpnet_fmap.fit(train_dl)

    report.append(evaluate_representation(dpnet_fmap, logmap))

GPU available: True (cuda), used: True
TPU available: False, using: 0 TPU cores
IPU available: False, using: 0 IPUs
HPU available: False, using: 0 HPUs
Seed set to 0
LOCAL_RANK: 0 - CUDA_VISIBLE_DEVICES: [0,1]


Fitting NNFeatureMap. Lookback window length set to 1


Training: |          | 0/? [00:00<?, ?it/s]

`Trainer.fit` stopped: `max_epochs=100` reached.
GPU available: True (cuda), used: True
TPU available: False, using: 0 TPU cores
IPU available: False, using: 0 IPUs
HPU available: False, using: 0 HPUs
Seed set to 1
LOCAL_RANK: 0 - CUDA_VISIBLE_DEVICES: [0,1]


Fitting NNFeatureMap. Lookback window length set to 1


...

Training: |          | 0/? [00:00<?, ?it/s]

`Trainer.fit` stopped: `max_epochs=100` reached.
GPU available: True (cuda), used: True
TPU available: False, using: 0 TPU cores
IPU available: False, using: 0 IPUs
HPU available: False, using: 0 HPUs
Seed set to 19
LOCAL_RANK: 0 - CUDA_VISIBLE_DEVICES: [0,1]


Fitting NNFeatureMap. Lookback window length set to 1


Training: |          | 0/? [00:00<?, ?it/s]

`Trainer.fit` stopped: `max_epochs=100` reached.


Now we calculate mean and standard deviation for each metrics:

In [83]:
def stack_reports(reports):
    stacked = {}
    for key in [
        "hausdorff-distance",
        "optimality-gap",
        # "feasibility-gap",
        # "fit-time",
        # "time_per_epoch",
    ]:
        if key in reports[0]:
            stacked[key] = np.mean([r[key] for r in reports])
            key_std = key + "_std"
            stacked[key_std] = np.std([r[key] for r in reports])
    # for key in ["estimator-eigenvalues", "covariance-eigenvalues"]:
    #     stacked[key] = np.stack([r[key] for r in reports])
    return stacked

In [84]:
full_DPNet_report = stack_reports(report)

In [85]:
full_DPNet_report

{'hausdorff-distance': 0.10157800004480881,
 'hausdorff-distance_std': 0.06901220238106452,
 'optimality-gap': 0.6838585513422009,
 'optimality-gap_std': 0.03076428073364825}

## Evaluating DPNet relaxed

Now we run the DPNet-relaxed over 20 independent runs. To do so we just need to change the flag that we pass to the DPNet module to whether use the relaxed version or not. For more information about the difference between the relaxed version of loss function with the original read the section 3 of the paper \cite{}.

In [86]:
DPNets = {'loss_fn': DPLoss, 'loss_kwargs': {'relaxed': True, 'metric_deformation': metric_deformation_coeff, 'center_covariances': False}}
report = []

for rng_seed in range(num_rng_seeds):

    trainer = lightning.Trainer(**trainer_kwargs)
    net_kwargs = {"feature_dim": feature_dim, "layer_dims": layer_dims}

    # Defining the model
    dpnet_relaxed_fmap = NNFeatureMap(
        SimpleMLP,
        DPNets['loss_fn'],
        opt,
        trainer,
        encoder_kwargs = net_kwargs,
        loss_kwargs = DPNets['loss_kwargs'],
        optimizer_kwargs = {"lr": 5e-5},
        lagged_encoder = SimpleMLP,
        lagged_encoder_kwargs = net_kwargs,
        seed = rng_seed)

    # Init
    torch.manual_seed(rng_seed)
    kaiming_init(dpnet_relaxed_fmap.lightning_module.encoder)
    kaiming_init(dpnet_relaxed_fmap.lightning_module.lagged_encoder)

    dpnet_relaxed_fmap.fit(train_dl)

    report.append(evaluate_representation(dpnet_relaxed_fmap, logmap))

GPU available: True (cuda), used: True
TPU available: False, using: 0 TPU cores
IPU available: False, using: 0 IPUs
HPU available: False, using: 0 HPUs
Seed set to 0
LOCAL_RANK: 0 - CUDA_VISIBLE_DEVICES: [0,1]


Fitting NNFeatureMap. Lookback window length set to 1


Training: |          | 0/? [00:00<?, ?it/s]

`Trainer.fit` stopped: `max_epochs=100` reached.
GPU available: True (cuda), used: True
TPU available: False, using: 0 TPU cores
IPU available: False, using: 0 IPUs
HPU available: False, using: 0 HPUs
Seed set to 1
LOCAL_RANK: 0 - CUDA_VISIBLE_DEVICES: [0,1]


Fitting NNFeatureMap. Lookback window length set to 1


...

Training: |          | 0/? [00:00<?, ?it/s]

`Trainer.fit` stopped: `max_epochs=100` reached.
GPU available: True (cuda), used: True
TPU available: False, using: 0 TPU cores
IPU available: False, using: 0 IPUs
HPU available: False, using: 0 HPUs
Seed set to 19
LOCAL_RANK: 0 - CUDA_VISIBLE_DEVICES: [0,1]


Fitting NNFeatureMap. Lookback window length set to 1


Training: |          | 0/? [00:00<?, ?it/s]

`Trainer.fit` stopped: `max_epochs=100` reached.


In [87]:
full_DPNet_relaxed_report = stack_reports(report)

In [88]:
full_DPNet_relaxed_report

{'hausdorff-distance': 0.12604366225348734,
 'hausdorff-distance_std': 0.09150072218359583,
 'optimality-gap': 1.2495531105412805,
 'optimality-gap_std': 0.08116260306880971}

## Evaluating VAMPNet

Now we run the VAMPNet \cite{} over 20 independent runs. To do so we just need to import the VAMPNet class from the kooplearn's feature maps.

In [90]:
from kooplearn.nn import VAMPLoss

In [91]:
VAMPNets = {'loss_fn': VAMPLoss, 'loss_kwargs': {'schatten_norm': 2, 'center_covariances': False}}
report = []

for rng_seed in range(num_rng_seeds):

    trainer = lightning.Trainer(**trainer_kwargs)
    net_kwargs = {"feature_dim": feature_dim, "layer_dims": layer_dims}

    # Defining the model
    vampnet_fmap = NNFeatureMap(
        SimpleMLP,
        VAMPNets['loss_fn'],
        opt,
        trainer,
        encoder_kwargs = net_kwargs,
        loss_kwargs = VAMPNets['loss_kwargs'],
        optimizer_kwargs = {"lr": 5e-5},
        lagged_encoder = SimpleMLP,
        lagged_encoder_kwargs = net_kwargs,
        seed = rng_seed)

    # Init
    torch.manual_seed(rng_seed)
    kaiming_init(vampnet_fmap.lightning_module.encoder)
    kaiming_init(vampnet_fmap.lightning_module.lagged_encoder)

    vampnet_fmap.fit(train_dl)

    report.append(evaluate_representation(vampnet_fmap, logmap))

GPU available: True (cuda), used: True
TPU available: False, using: 0 TPU cores
IPU available: False, using: 0 IPUs
HPU available: False, using: 0 HPUs
Seed set to 0
LOCAL_RANK: 0 - CUDA_VISIBLE_DEVICES: [0,1]


Fitting NNFeatureMap. Lookback window length set to 1


Training: |          | 0/? [00:00<?, ?it/s]

`Trainer.fit` stopped: `max_epochs=100` reached.
GPU available: True (cuda), used: True
TPU available: False, using: 0 TPU cores
IPU available: False, using: 0 IPUs
HPU available: False, using: 0 HPUs
Seed set to 1
LOCAL_RANK: 0 - CUDA_VISIBLE_DEVICES: [0,1]


Fitting NNFeatureMap. Lookback window length set to 1


...

Training: |          | 0/? [00:00<?, ?it/s]

`Trainer.fit` stopped: `max_epochs=100` reached.
GPU available: True (cuda), used: True
TPU available: False, using: 0 TPU cores
IPU available: False, using: 0 IPUs
HPU available: False, using: 0 HPUs
Seed set to 19
LOCAL_RANK: 0 - CUDA_VISIBLE_DEVICES: [0,1]


Fitting NNFeatureMap. Lookback window length set to 1


Training: |          | 0/? [00:00<?, ?it/s]

`Trainer.fit` stopped: `max_epochs=100` reached.


In [92]:
full_VAMPNet_report = stack_reports(report)

In [93]:
full_VAMPNet_report

{'hausdorff-distance': 0.2514556783788494,
 'hausdorff-distance_std': 0.1070655612172945,
 'optimality-gap': 0.9564874527305818,
 'optimality-gap_std': 0.14154826095376252}

## Evaluating Noise Kernel

In [94]:
import scipy.special
from kooplearn.models.feature_maps import ConcatenateFeatureMaps

In [95]:
def noise_feat(n, x):
    return logmap.noise_feature(x, n)

def NoiseKernel(order: int = 3):
    binom_coeffs = [scipy.special.binom(N_exp, i) for i in range(N_exp + 1)]
    sorted_coeffs = np.argsort(binom_coeffs)

    fn_list = [partial(noise_feat, n) for n in sorted_coeffs[:order]]

    return ConcatenateFeatureMaps(fn_list)

In [97]:
full_NoiseKernel_report = evaluate_representation(NoiseKernel(feature_dim), logistic_map=logmap)

In [98]:
full_NoiseKernel_report

{'hausdorff-distance': 0.1923270520720579,
 'optimality-gap': 2.1733452667555846}

## Evaluating Cheby-T Kernel

In [99]:
def scaled_chebyt(n, x):
    return scipy.special.eval_chebyt(n, 2 * x - 1)

def ChebyT(feature_dim: int = 3):

    fn_list = [partial(scaled_chebyt, n) for n in range(feature_dim)]
    return ConcatenateFeatureMaps(fn_list)

In [100]:
full_ChebyTKenel_report = evaluate_representation(ChebyT(feature_dim),logmap)

In [101]:
full_ChebyTKenel_report

{'hausdorff-distance': 0.2020483781440318,
 'optimality-gap': 1.2401792555398874}